In [ ]:
import time 
import re
import ctypes
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import  expected_conditions as EC 
from selenium.common.exceptions import TImeoutException, NoSuchElementException, taleElementReferenceException

In [ ]:
ctypes.windll.kernal32.SetThreadExecutionState(x80000002) # stops sleep (windows)

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get('https://www.laliga.com/en-GB/laliga-easports/clubs')

retry_attempts = 0
max_retries = 1

while retry_attempts <= max_retries:
    try:
        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'link'))
        ) 
        club_links = driver.find_elements(By.CLASS_NAME, 'link')
        break
    except TimeoutException:
        retry_attempts += 1
        print(f"Timeout exception occured. Attempt {retry_attempts} of {max_retries + 1}")
        if retry_attempts > max_retries:
            print('Max retries reached. Stopping script')

<a class="link" target="_self" globaldata="[object Object]" datachangelang="[object Object]" gptbase="/laliga-easports/clubes" href="/en-GB/clubs/athletic-club/squad"><div class="styled__AvatarContainer-sc-fyva03-2 fpOkUs"><div class="styled__AvatarContainer-sc-1cljep8-0 kTyNMd">

ok so looks like every club link has the class link so i should target that.

try:
    # Wait for the elements
    wait = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'link'))
    )
    # Now fetch the elements once they’re confirmed present
    club_links = driver.find_elements(By.CLASS_NAME, 'link')

except TimeoutException:
    print('There has been a timeout exception')
    # Optional: Log to a file or retry logic